In [1]:
# =========================
# IMPORTS
# =========================

from X_functions import *
from X_config import *
from X_models import *
from X_interpolation import *
from X_plots import *

from GGS_main import *

# =========================

Cannot find header.dxf (GDAL_DATA is not defined)
c:\Users\sal_f\anaconda3\envs\env_ggs0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

### GGS Executable

In [2]:
power=1
path="local"
config_name="test"

In [3]:
if config_name is None:
    print("No config file specified. Exiting.")
    exit(1)

config = GGS_config_import(config_name)

target_datetime = config['MISSION'].get('target_date')
if not target_datetime:
    print("Issue with target datetime. Using current datetime.")
    target_datetime = dt.datetime.now(dt.timezone.utc)

if path == "local":
    root_directory = GGS_config_process(config, path="default")
elif path == "rucool":
    root_directory = GGS_config_process(config, path="/www/web/rucool/hurricane/model_comparisons/maps/yucatan")
else:
    raise ValueError("Invalid root directory.")

datetime_list = [target_datetime.replace(hour=0, minute=0, second=0, microsecond=0).strftime('%Y-%m-%dT%H:%M:%SZ')]
datetime_index = datetime_list[0]

glider_dataframes = None
if config['PLOT'].get('show_gliders'):
    min_lat, min_lon = config['MISSION']['extent'][0]
    max_lat, max_lon = config['MISSION']['extent'][1]
    search_extent = [min_lon, max_lon, min_lat, max_lat]
    glider_dataframes = acquire_gliders(
        extent=search_extent,
        target_date=target_datetime,
        date_delta=dt.timedelta(days=1),
        requested_variables=["time", "longitude", "latitude", "profile_id", "depth"],
        print_vars=False,
        target="all",
        request_timeout=5,
        enable_parallel=False
    )


### IMPORTING GGS CONFIGURATION: test ###

Configuration import success!

### PROCESSING GGS CONFIGURATION ###

Configuration:

--> MISSION
mission_name: TEST
target_date: 2024-04-07T15:19:24+0000
max_depth: 1000
extent: ((-40, 0), (-20, 22))
GPS_coords: [[15.067, -23.65], [-33.907, 18.564]]
glider_id: None

--> MODEL
single_datetime: True
enable_rtofs: False
enable_cmems: True
enable_gofs: True
save_model_data: False
save_depth_average: True
save_bin_average: False
chunk: True

--> PLOT
latitude_qc: -6
longitude_qc: 0
density: 3
mag1: 0.0
mag2: 0.2
mag3: 0.3
mag4: 0.4
mag5: 0.5
tolerance: 7.5
show_gliders: False
show_route: True
show_eez: True
show_qc: False
manual_extent: None

--> DATA
bathymetry_path: c:\Users\sal_f\OneDrive\Desktop\STF-0\!-GGS\GGS_Scripts\data/bathymetry/GEBCO_2023_sub_ice_topo.nc
eez_path: c:\Users\sal_f\OneDrive\Desktop\STF-0\!-GGS\GGS_Scripts\data/eez/eez_boundaries_v12.shp

--> ADVANCED
reprocess: False



In [4]:
datetime_index = datetime_index
config_flag = config
root_directory_flag = root_directory
glider_data_flag = glider_dataframes

enable_rtofs_flag = config_flag['MODEL']['enable_rtofs']
enable_cmems_flag = config_flag['MODEL']['enable_cmems']
enable_gofs_flag = config_flag['MODEL']['enable_gofs']
save_model_data_flag = config_flag['MODEL']['save_model_data']
save_depth_average_flag = config_flag['MODEL']['save_depth_average']
save_bin_average_flag = config_flag['MODEL']['save_bin_average']
chunk_flag = config_flag['MODEL']['chunk']

latitude_qc_flag = config_flag['PLOT']['latitude_qc']
longitude_qc_flag = config_flag['PLOT']['longitude_qc']
density_flag = config_flag['PLOT']['density']
mag1_flag = config_flag['PLOT']['mag1']
mag2_flag = config_flag['PLOT']['mag2']
mag3_flag = config_flag['PLOT']['mag3']
mag4_flag = config_flag['PLOT']['mag4']
mag5_flag = config_flag['PLOT']['mag5']
tolerance_flag = config_flag['PLOT']['tolerance']
show_route_flag = config_flag['PLOT']['show_route']
show_eez_flag = config_flag['PLOT']['show_eez']
show_qc_flag = config_flag['PLOT']['show_qc']
manual_extent_flag = config_flag['PLOT']['manual_extent']

sub_directory_plots = os.path.join(root_directory_flag, "plots", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_plots, exist_ok=True)
sub_directory_data = os.path.join(root_directory_flag, "data", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_data, exist_ok=True)

check_datetime = pd.to_datetime(datetime_index).strftime('%Y%m%dT%HZ')
check_pattern = os.path.join(sub_directory_data, f"*_DepthAverageData_{check_datetime}.nc")
check_files = glob.glob(check_pattern)
if check_files:
    print(f"Datetime {datetime_index} already processed: {check_files[0]}, skipping task.")
else:
    print(f"Datetime {datetime_index} unprocessed, proceeding with task.")

Datetime 2024-04-07T00:00:00Z unprocessed, proceeding with task.


In [5]:
model_datasets = []
    
if enable_rtofs_flag:
    try:
        rtofs = RTOFS()
        rtofs.rtofs_load(config_flag, datetime_index)
        rtofs.rtofs_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
        rtofs_model_data = rtofs.data
        
        rtofs_depth_average, rtofs_bin_average = interpolate_rtofs(config_flag, sub_directory_data, rtofs_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)

        rtofs_datasets = (rtofs_model_data, rtofs_depth_average, rtofs_bin_average)
        model_datasets.append(rtofs_datasets)
    except Exception as e:
        print(f"Error during RTOFS processing: {e}")

if enable_cmems_flag:
    try:
        cmems = CMEMS(username='sfricano1', password='GlobalGliders1')
        cmems.cmems_load(config_flag, datetime_index)
        cmems.cmems_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
        cmems_model_data = cmems.data
        
        cmems_depth_average, cmems_bin_average = interpolate_cmems(config_flag, sub_directory_data, cmems_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)
        
        cmems_datasets = (cmems_model_data, cmems_depth_average, cmems_bin_average)
        model_datasets.append(cmems_datasets)
    except Exception as e:
        print(f"Error during CMEMS processing: {e}")

if enable_gofs_flag:
    try:
        gofs = GOFS()
        gofs.gofs_load(config_flag, datetime_index)
        gofs.gofs_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
        gofs_model_data = gofs.data

        gofs_depth_average, gofs_bin_average = interpolate_gofs(config_flag, sub_directory_data, gofs_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)

        gofs_datasets = (gofs_model_data, gofs_depth_average, gofs_bin_average)
        model_datasets.append(gofs_datasets)
    except Exception as e:
        print(f"Error during GOFS processing: {e}")


### MODEL DATA: [CMEMS] ###

INFO - 2024-04-07T15:19:28Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-04-07T15:19:28Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-04-07T15:19:30Z - Service was not specified, the default one was selected: "arco-geo-series"

### INTERPOLATING CMEMS MODEL DATA ###

Start time (UTC): 2024-04-07 15:19:33.340688
End time (UTC): 2024-04-07 15:20:04.592568
Run time: 0:00:31.251880

### MODEL DATA: [GOFS] ###


### INTERPOLATING GOFS MODEL DATA ###

Start time (UTC): 2024-04-07 15:20:05.620961
End time (UTC): 2024-04-07 15:21:37.327145
Run time: 0:01:31.706184


In [ ]:
GGS_plot_magnitude(
    config_flag,
    sub_directory_plots,
    datetime_index,
    model_datasets,
    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
    density=density_flag,
    gliders=glider_data_flag,
    show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
    manual_extent=manual_extent_flag
)
GGS_plot_threshold(
    config_flag,
    sub_directory_plots,
    datetime_index,
    model_datasets,
    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
    density=density_flag,
    mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
    gliders=glider_data_flag,
    show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
    manual_extent=manual_extent_flag
)
GGS_plot_advantage(
    config_flag,
    sub_directory_plots,
    datetime_index,
    model_datasets,
    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
    density=density_flag,
    tolerance=tolerance_flag,
    mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
    gliders=glider_data_flag,
    show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
    manual_extent=manual_extent_flag
)
GGS_plot_profiles(
    config_flag,
    sub_directory_plots,
    datetime_index,
    model_datasets,
    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
    threshold=0.5
)

---

### GGS Testing

In [ ]:
# cmems_depth_average = xr.open_dataset("C:/Users/sal_f/OneDrive/Desktop/STF-0/!-GGS/GGS_Scripts/data/reprocess/Sentinel Leg 2_CMEMS_DepthAverage_20240328T00Z.nc")

In [10]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

def GGS_export_gpkg(directory, datetime_index, model_datasets):
    
    '''
    Process and export data from model datasets to CSV and GeoPackage files.

    Args:
    - directory (str): Directory to save the files.
    - datetime_index (str): Datetime index for the model datasets.
    - model_datasets (tuple): Tuple containing the model datasets.

    Returns:
    - None
    '''

    print(f"\n### CREATING GEODATAFRAME FILES ###\n")
    start_time = print_starttime()

    valid_datasets = [datasets for datasets in model_datasets if datasets is not None]
    num_datasets = len(valid_datasets)
    if num_datasets == 0:
        print("No datasets provided for GeoDataFrame conversion.")
        end_time = print_endtime()
        print_runtime(start_time, end_time)
        return

    file_datetime = format_save_datetime(datetime_index)
    for model_data, depth_average_data, bin_average_data in valid_datasets:
        model_name = depth_average_data.attrs['model_name']
        csv_file = f"{model_name}_depth_average_{file_datetime}.csv"
        csv_path = os.path.join(directory, csv_file)
        gpkg_file = f"{model_name}_depth_average_{file_datetime}.gpkg"
        gpkg_path = os.path.join(directory, gpkg_file)

        dataframe = depth_average_data.to_dataframe().reset_index()
        dataframe = dataframe[['lat', 'lon', 'mag_depth_avg', 'dir_depth_avg']]
        dataframe.dropna(subset=['mag_depth_avg', 'dir_depth_avg'], inplace=True)
        dataframe.to_csv(csv_path, index=False)

        geometry = [Point(xy) for xy in zip(dataframe['lon'], dataframe['lat'])]
        geodataframe = gpd.GeoDataFrame(dataframe, geometry=geometry)
        geodataframe.crs = "EPSG:4326"
        geodataframe.to_file(gpkg_path, driver="GPKG")

    end_time = print_endtime()
    print_runtime(start_time, end_time)

GGS_export_gpkg(config, sub_directory_data, model_datasets)


### CREATING GEODATAFRAME FILES ###

Start time (UTC): 2024-04-07 15:46:46.752521
End time (UTC): 2024-04-07 15:47:02.861882
Run time: 0:00:16.109361


---